In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st

In [2]:
df = pd.read_csv('spotify_for_ml.csv')

## Категориальные фичи (фича)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16695 entries, 0 to 16694
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist            16695 non-null  object 
 1   track             16694 non-null  object 
 2   album             16694 non-null  object 
 3   album_type        16694 non-null  object 
 4   danceability      16694 non-null  float64
 5   energy            16694 non-null  float64
 6   key               16694 non-null  float64
 7   loudness          16694 non-null  float64
 8   speechiness       16694 non-null  float64
 9   acousticness      16694 non-null  float64
 10  instrumentalness  16694 non-null  float64
 11  liveness          16694 non-null  float64
 12  valence           16694 non-null  float64
 13  tempo             16694 non-null  float64
 14  stream            16694 non-null  float64
 15  duration_min      16694 non-null  float64
 16  is_feat           16694 non-null  float6

Используя выводы, полученные на предыдущем этапе работы о типе релиза, учтем в модели признак is_album, который будет отвечать за тип релиза, в составе которого участвовал трек:
1. **Альбом** → is_album = True (1)
2. **Сингл** → is_album = False (0)

In [4]:
df.album_type.unique()

array(['album', 'single', nan], dtype=object)

In [5]:
df['is_album'] = [1 if a == 'album' else 0 for a in df.album_type]

Уберем признаки artist, track, album, album_type:

In [6]:
df = df.drop(['artist', 'track', 'album', 'album_type'], axis = 1)
df.shape

(16695, 14)

## Линейная регрессия

Перед обучением модели сделаем следующее:

1. Разделим датафрейм на датафрейм с признаками - Х и массив с целевой переменной - у (количество прослушиваний)

2. После этого разобъем датафрейм на train и test в соотношении 80 / 20. Также логарифмируем целевую переменную, чтобы нормализовать ее распределение

In [7]:
from sklearn.model_selection import train_test_split
import numpy

X = df.dropna().copy().drop(['stream'], axis = 1)

#y = df['stream'].copy()
log_y = df.dropna()['stream'].copy().apply(np.log)

X_train, X_test, log_y_train, log_y_test = train_test_split(X, log_y, test_size = 0.2, random_state = 42)

Стандартизируем признаки:

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 1. Для начала попробуем просто построить линейную регрессию "как есть", добавив столбец константы:

In [9]:
# Добавляем константу:
X_train_scaled_b = np.concatenate([np.ones((X_train.shape[0], 1)), X_train_scaled], axis=1)
X_test_scaled_b = np.concatenate([np.ones((X_test.shape[0], 1)), X_test_scaled], axis=1)

# Обучаем модель:
from sklearn.linear_model import LinearRegression

reg1 = LinearRegression(fit_intercept = False).fit(X_train_scaled_b, log_y_train)

log_y_train_pred = reg1.predict(X_train_scaled_b)
log_y_test_pred = reg1.predict(X_test_scaled_b)

from sklearn.metrics import r2_score, mean_squared_error

r2_train = r2_score(log_y_train, log_y_train_pred)
r2_test = r2_score(log_y_test, log_y_test_pred)

mse_train = mean_squared_error(log_y_train, log_y_train_pred)
mse_test = mean_squared_error(log_y_test, log_y_test_pred)

#Вывод
coef_table = pd.DataFrame({'Признак': ['const'] + list(X_train.columns), 
                           'Коэффициент': list(reg1.coef_.round(4))})

print(coef_table)

print('--------------------------------')
print(f'R² на train: {round(r2_train * 100, 2)}%')
print(f'R² на test: {round(r2_test * 100, 2)}%')
print(f'MSE на train: {round(mse_train, 4)}')
print(f'MSE на test: {round(mse_test, 4)}')

             Признак  Коэффициент
0              const      17.7323
1       danceability       0.0559
2             energy      -0.2340
3                key       0.0050
4           loudness       0.2950
5        speechiness      -0.0853
6       acousticness      -0.1222
7   instrumentalness      -0.0784
8           liveness      -0.0580
9            valence      -0.0762
10             tempo       0.0087
11      duration_min       0.0772
12           is_feat       0.0945
13          is_album       0.3361
--------------------------------
R² на train: 7.99%
R² на test: 7.93%
MSE на train: 2.4174
MSE на test: 2.4307


R² крайне низкий, на уровне 7,9%-8%

**Проверим данные на наличие мультиколлинеарности** - для этого посчитаем Variance Inflation Factor (VIF) для регрессоров в модели:

In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def vif_coeff(df):
    vif = pd.DataFrame()
    vif['feature'] = df.columns
    vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif

vif = vif_coeff(X_train) 
print(vif)

             feature        VIF
0       danceability  19.108190
1             energy  23.454353
2                key   3.155738
3           loudness   9.572215
4        speechiness   2.208602
5       acousticness   2.844166
6   instrumentalness   1.178048
7           liveness   2.450608
8            valence   8.977475
9              tempo  16.758763
10      duration_min  18.567560
11           is_feat   1.184559
12          is_album   4.404166


**Коэффициенты VIF больше 10**, значит скорее всего в модели присутствует мультиколлинеарность
https://books.econ.msu.ru/Introduction-to-Econometrics/chap04/4.1/?ysclid=mbwgropa3o744583280

### 2. Попробуем убрать признаки, для которых VIF > 10:

In [11]:
drop_cols = []
for i in range(vif.shape[0]):
    if vif.VIF[i] > 10:
        drop_cols.append(vif.feature[i])
drop_cols

['danceability', 'energy', 'tempo', 'duration_min']

In [12]:
X_train2 = X_train.drop(drop_cols, axis = 1)
X_test2 = X_test.drop(drop_cols, axis = 1)

# Стандартизируем
scaler = StandardScaler().fit(X_train2)
X_train_scaled2 = scaler.transform(X_train2)
X_test_scaled2 = scaler.transform(X_test2)

# Добавляем константу:
X_train_scaled_b2 = np.concatenate([np.ones((X_train2.shape[0], 1)), X_train_scaled2], axis = 1)
X_test_scaled_b2 = np.concatenate([np.ones((X_test2.shape[0], 1)), X_test_scaled2], axis = 1)

# Обучаем модель:
from sklearn.linear_model import LinearRegression

reg2 = LinearRegression(fit_intercept = False).fit(X_train_scaled_b2, log_y_train)

log_y_train_pred2 = reg2.predict(X_train_scaled_b2)
log_y_test_pred2 = reg2.predict(X_test_scaled_b2)

from sklearn.metrics import r2_score, mean_squared_error

r2_train = r2_score(log_y_train, log_y_train_pred2)
r2_test = r2_score(log_y_test, log_y_test_pred2)

mse_train = mean_squared_error(log_y_train, log_y_train_pred2)
mse_test = mean_squared_error(log_y_test, log_y_test_pred2)


#Вывод
coef_table = pd.DataFrame({'Признак': ['const'] + list(X_train2.columns), 
                           'Коэффициент': list(reg2.coef_.round(4))})

print(coef_table)

print('--------------------------------')
print(f'R² на train: {round(r2_train * 100, 2)}%')
print(f'R² на test: {round(r2_test * 100, 2)}%')
print(f'MSE на train: {round(mse_train, 4)}')
print(f'MSE на test: {round(mse_test, 4)}')

            Признак  Коэффициент
0             const      17.7323
1               key       0.0042
2          loudness       0.1642
3       speechiness      -0.0843
4      acousticness      -0.0575
5  instrumentalness      -0.1093
6          liveness      -0.0885
7           valence      -0.1104
8           is_feat       0.1049
9          is_album       0.3412
--------------------------------
R² на train: 7.0%
R² на test: 6.79%
MSE на train: 2.4436
MSE на test: 2.4607


R² снизился, что может говорить о том, что мы убрали важный признак, объясняющий часть дисперсии целевой переменной. 

### 3. Попробуем не убирать сразу все признаки с VIF > 10:

In [13]:
def metrics_linreg(X_train, X_test, log_y_train, log_y_test):
    
    #Стандартизируем
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    #Добавляем константу
    X_train_scaled_b = np.concatenate([np.ones((X_train.shape[0], 1)), X_train_scaled], axis=1)
    X_test_scaled_b = np.concatenate([np.ones((X_test.shape[0], 1)), X_test_scaled], axis=1)
    
    #Обучаем модель
    from sklearn.linear_model import LinearRegression    
    reg = LinearRegression(fit_intercept = False).fit(X_train_scaled_b, log_y_train)

    log_y_train_pred = reg.predict(X_train_scaled_b)
    log_y_test_pred = reg.predict(X_test_scaled_b)

    from sklearn.metrics import r2_score, mean_squared_error

    r2_train = r2_score(log_y_train, log_y_train_pred)
    r2_test = r2_score(log_y_test, log_y_test_pred)
    
    mse_train = mean_squared_error(log_y_train, log_y_train_pred)
    mse_test = mean_squared_error(log_y_test, log_y_test_pred)

    #Вывод
    print('--------------------------------')
    print(f'R² на train: {round(r2_train * 100, 2)}%')
    print(f'R² на test: {round(r2_test * 100, 2)}%')
    print(f'MSE на train: {round(mse_train, 4)}')
    print(f'MSE на test: {round(mse_test, 4)}')
    print('--------------------------------')

In [14]:
cols = ['danceability', 'energy', 'tempo', 'duration_min']

combs = ['0', '1', '2', '3', 
         '01', '02', '03', 
         '12', '13', '23', 
         '012', '013', '023', '123']

for comb in combs:
    drop_cols = []
    
    for i in comb:
        drop_cols.append(cols[int(i)])
        
    X_train_viftest = X_train.drop(drop_cols, axis = 1)
    X_test_viftest = X_test.drop(drop_cols, axis = 1)
    
    print('--------------------------------')
    print(f'Убрали признаки: {drop_cols}')
    metrics_linreg(X_train_viftest, X_test_viftest, log_y_train, log_y_test)


--------------------------------
Убрали признаки: ['danceability']
--------------------------------
R² на train: 7.91%
R² на test: 7.93%
MSE на train: 2.4195
MSE на test: 2.4307
--------------------------------
--------------------------------
Убрали признаки: ['energy']
--------------------------------
R² на train: 7.34%
R² на test: 7.06%
MSE на train: 2.4345
MSE на test: 2.4536
--------------------------------
--------------------------------
Убрали признаки: ['tempo']
--------------------------------
R² на train: 7.99%
R² на test: 7.91%
MSE на train: 2.4174
MSE на test: 2.4311
--------------------------------
--------------------------------
Убрали признаки: ['duration_min']
--------------------------------
R² на train: 7.78%
R² на test: 7.61%
MSE на train: 2.4229
MSE на test: 2.439
--------------------------------
--------------------------------
Убрали признаки: ['danceability', 'energy']
--------------------------------
R² на train: 7.14%
R² на test: 6.99%
MSE на train: 2.4398
MS

Максимальный R² получился, когда был убран только признак ***tempo***

При этом R² и MSE совпадают со значениями до удаления признаков, что говорит о том, что он не влияет на качество модели и вероятно не имеет значимого влияния на целевую переменную. 

В контексте данных, с которыми мы работаем, можно предположить, что *базовые* свойства композиций (длительность, темп) "зашиты" в *производные* характеристики треков - energy, danceability, liveness и др.

Поскольку VIF у 4 переменных сильно высокий, а значительного прироста в качестве не наблюдается, оставим только базовые параметры в целях улучшения интерпретируемости и сохранения потенциально важной информации о композициях:

**Оставляем:**
* duration_min
* tempo

**Убираем:**
* danceability
* energy

**При прочих равных характеристики модели:**
* R² на train: 7.14%
* R² на test: 6.99%
* MSE на train: 2.4398
* MSE на test: 2.4554

### 4. Попробуем использовать Регуляризацию:

In [15]:
def reg_linereg(X_train, X_test, log_y_train, log_y_test, alpha, reg_type, drop_cols):
    
    from sklearn.linear_model import Ridge, Lasso
    
    X_train_r = X_train.drop(drop_cols, axis = 1)
    X_test_r = X_test.drop(drop_cols, axis = 1)

    # Стандартизируем
    scaler = StandardScaler().fit(X_train_r)
    X_train_scaled_r = scaler.transform(X_train_r)
    X_test_scaled_r = scaler.transform(X_test_r)

    # Добавляем константу:
    X_train_scaled_b_r = np.concatenate([np.ones((X_train_r.shape[0], 1)), X_train_scaled_r], axis = 1)
    X_test_scaled_b_r = np.concatenate([np.ones((X_test_r.shape[0], 1)), X_test_scaled_r], axis = 1)


    # Обучаем:
    if reg_type == 'l1':
        model = Lasso(alpha = alpha, fit_intercept = False).fit(X_train_scaled_b_r, log_y_train)
    else:
        model = Ridge(alpha = alpha, fit_intercept = False).fit(X_train_scaled_b_r, log_y_train)
    
    log_y_train_pred = model.predict(X_train_scaled_b_r)
    log_y_test_pred = model.predict(X_test_scaled_b_r)


    # Выводим:
    r2_train = r2_score(log_y_train, log_y_train_pred)
    r2_test = r2_score(log_y_test, log_y_test_pred)
    
    mse_train = mean_squared_error(log_y_train, log_y_train_pred)
    mse_test = mean_squared_error(log_y_test, log_y_test_pred)
    print('--------------------------------')
    print(f'Alpha = {alpha}')
    print('--------------------------------')
    print(f'R² на train: {round(r2_train * 100, 2)}%')
    print(f'R² на test: {round(r2_test * 100, 2)}%')
    print(f'MSE на train: {round(mse_train, 4)}')
    print(f'MSE на test: {round(mse_test, 4)}')
    print('--------------------------------')

**Посмотрим, как меняются показатели моделей при разных alpha:**

In [16]:
# Lasso (L1)
print('L1 регуляризация (Lasso)')
for alpha in (0.1, 0.5, 1, 10, 50, 100):
    reg_linereg(X_train, X_test, log_y_train, log_y_test, alpha, reg_type = 'l1', drop_cols = ['danceability', 'energy'])

print('################################')
print('--------------------------------')
# Ridge (L2)
print('L2 регуляризация (Ridge)')
for alpha in (0.1, 0.5, 1, 10, 50, 100):
    reg_linereg(X_train, X_test, log_y_train, log_y_test, alpha, reg_type = 'l2', drop_cols = ['danceability', 'energy'])

L1 регуляризация (Lasso)
--------------------------------
Alpha = 0.1
--------------------------------
R² на train: 4.1%
R² на test: 4.26%
MSE на train: 2.5198
MSE на test: 2.5274
--------------------------------
--------------------------------
Alpha = 0.5
--------------------------------
R² на train: -9.52%
R² на test: -8.92%
MSE на train: 2.8774
MSE на test: 2.8755
--------------------------------
--------------------------------
Alpha = 1
--------------------------------
R² на train: -38.06%
R² на test: -36.77%
MSE на train: 3.6274
MSE на test: 3.6108
--------------------------------
--------------------------------
Alpha = 10
--------------------------------
R² на train: -3806.05%
R² на test: -3776.81%
MSE на train: 102.6274
MSE на test: 102.3462
--------------------------------
--------------------------------
Alpha = 50
--------------------------------
R² на train: -11967.45%
R² на test: -11890.79%
MSE на train: 317.0602
MSE на test: 316.5517
--------------------------------
---

**Выводы:**
1. При использовании Lasso с alpha > 0.1 модель активно зануляет признаки, из-за чего R² становится отрицательным - модель работает хуже, чем предсказание по средним значениям
2. При использовании Ridge с alpha <= 10 значительных изменений в R² нет

### Полиномиальные фичи

Попробуем добавить полиномиальные фичи в модель с регуляризацией, чтобы проработать возможную нелинейность взаимосвязей с целевой переменной:

In [17]:
# Возьмем изначальные данные:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 2, include_bias = False)

cols = X_train.columns

train_poly = poly.fit_transform(X_train)
test_poly = poly.transform(X_test)

X_train_poly = pd.DataFrame(train_poly, columns = poly.get_feature_names_out())
X_test_poly = pd.DataFrame(test_poly, columns = poly.get_feature_names_out())

In [18]:
# Lasso (L1)
print('L1 регуляризация (Lasso)')
for alpha in (0.1, 0.5, 1, 10, 50, 100):
    reg_linereg(X_train_poly, X_test_poly, log_y_train, log_y_test, alpha, reg_type = 'l1', drop_cols = [])

print('################################')
print('--------------------------------')
# Ridge (L2)
print('L2 регуляризация (Ridge)')
for alpha in (0.1, 0.5, 1, 10, 50, 100):
    reg_linereg(X_train_poly, X_test_poly, log_y_train, log_y_test, alpha, reg_type = 'l2', drop_cols = [])

L1 регуляризация (Lasso)
--------------------------------
Alpha = 0.1
--------------------------------
R² на train: 5.46%
R² на test: 5.34%
MSE на train: 2.484
MSE на test: 2.4989
--------------------------------
--------------------------------
Alpha = 0.5
--------------------------------
R² на train: -9.52%
R² на test: -8.92%
MSE на train: 2.8774
MSE на test: 2.8755
--------------------------------
--------------------------------
Alpha = 1
--------------------------------
R² на train: -38.06%
R² на test: -36.77%
MSE на train: 3.6274
MSE на test: 3.6108
--------------------------------
--------------------------------
Alpha = 10
--------------------------------
R² на train: -3806.05%
R² на test: -3776.81%
MSE на train: 102.6274
MSE на test: 102.3462
--------------------------------
--------------------------------
Alpha = 50
--------------------------------
R² на train: -11967.45%
R² на test: -11890.79%
MSE на train: 317.0602
MSE на test: 316.5517
--------------------------------
---

**С точки зрения R² лучше справилась Ridge.**

Возьмем модель с alpha = 50 и посмотрим на коэффициенты:

In [19]:
from sklearn.linear_model import Ridge

# Стандартизируем
scaler = StandardScaler().fit(X_train_poly)
X_train_scaled_r = scaler.transform(X_train_poly)
X_test_scaled_r = scaler.transform(X_test_poly)

# Добавляем константу:
X_train_scaled_b_r = np.concatenate([np.ones((X_train.shape[0], 1)), X_train_scaled_r], axis = 1)
X_test_scaled_b_r = np.concatenate([np.ones((X_test.shape[0], 1)), X_test_scaled_r], axis = 1)


# Обучаем:
alpha = 50
model = Ridge(alpha = alpha, fit_intercept = False).fit(X_train_scaled_b_r, log_y_train)
    
log_y_train_pred = model.predict(X_train_scaled_b_r)
log_y_test_pred = model.predict(X_test_scaled_b_r)


# Выводим:
r2_train = r2_score(log_y_train, log_y_train_pred)
r2_test = r2_score(log_y_test, log_y_test_pred)
    
mse_train = mean_squared_error(log_y_train, log_y_train_pred)
mse_test = mean_squared_error(log_y_test, log_y_test_pred)

coef_table = pd.DataFrame({'Признак': ['const'] + list(X_train_poly.columns), 
                           'Коэффициент': list(model.coef_.round(4))})

pd.set_option('display.max_rows', None)
print(coef_table.sort_values(by = 'Коэффициент', ascending = False).reset_index(drop = True))
pd.reset_option('display.max_rows')


print('-----------------------------------------------')
print(f'Alpha = {alpha}')
print('-----------------------------------------------')
print(f'R² на train: {round(r2_train * 100, 2)}%')
print(f'R² на test: {round(r2_test * 100, 2)}%')
print(f'MSE на train: {round(mse_train, 4)}')
print(f'MSE на test: {round(mse_test, 4)}')
print('-----------------------------------------------')

                           Признак  Коэффициент
0                            const      17.6661
1                     duration_min       0.6192
2                  energy is_album       0.3768
3            danceability loudness       0.2782
4                   energy valence       0.2570
5                         loudness       0.2464
6                   danceability^2       0.2041
7               tempo duration_min       0.1777
8                speechiness tempo       0.1708
9             duration_min is_feat       0.1686
10       danceability acousticness       0.1597
11                    danceability       0.1590
12              danceability tempo       0.1549
13                      loudness^2       0.1370
14        speechiness duration_min       0.1339
15                  loudness tempo       0.0993
16                instrumentalness       0.0964
17                    acousticness       0.0916
18             energy duration_min       0.0838
19            danceability is_feat      

### Выводы
1. **Наибольшее влияние имеют переменные duration_min (положительное) и duration_min^2 (отрицательное)**

С точки зрения интерпретации можно предположить, что при значительном увеличении продолжительности трека его популярность снижается

2. **Коэффициенты при loudness и loudness^2 положительны**

т.е. в среднем при более высокой громкости трека у него больше прослушиваний. Это может подтверждать предположение об использовании громкости в качестве прокси качества трека (т.к. для достижения более высокой громкости нужно более качественное сведение композиции)

3. **R² на test = 9.37%**, 

что очень мало и говорит о плохой объясняющей силе модели, поэтому ***выводы не являются надёжными.***

Возможно стоит попробовать более сложные типы моделей

## Бустинг

Попробуем использовать другие модели, например **LightGBM (*Light Gradient Boosting Machine*)** - ансамблевая модель, основанная на стохастическом градиентном бустинге

**Для начала используем изначальные данные:** 

In [20]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(n_estimators = 100, 
                     learning_rate = 0.1, 
                     max_depth = 3, 
                     random_state = 42)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

lgbm.fit(X_train_scaled, log_y_train)

log_y_train_pred_lgbm = lgbm.predict(X_train_scaled)
log_y_test_pred_lgbm = lgbm.predict(X_test_scaled)

r2_train = r2_score(log_y_train, log_y_train_pred_lgbm)
r2_test = r2_score(log_y_test, log_y_test_pred_lgbm)

rmse_train = mean_squared_error(log_y_train, log_y_train_pred_lgbm) ** 0.5
rmse_test = mean_squared_error(log_y_test, log_y_test_pred_lgbm) ** 0.5

print('--------------------------------')
print(f'R² на train: {round(r2_train * 100, 2)}%')
print(f'R² на test: {round(r2_test * 100, 2)}%')
print(f'RMSE на train: {round(rmse_train, 4)}')
print(f'RMSE на test: {round(rmse_test, 4)}')
print('--------------------------------')

--------------------------------
R² на train: 15.35%
R² на test: 9.87%
RMSE на train: 1.4913
RMSE на test: 1.5425
--------------------------------


Даже на изначальных данных модель показывает значительный прирост R², однако на train он сильно выше, чем на test, что может говорить о наличии переобучения.

**Добавим полиномиальные признаки, чтобы улучшить модель:**

In [21]:
lgbm = LGBMRegressor(n_estimators = 100, 
                     learning_rate = 0.1, 
                     max_depth = 3, 
                     random_state = 42)

scaler = StandardScaler().fit(X_train_poly)
X_train_scaled = scaler.transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

lgbm.fit(X_train_scaled, log_y_train)

log_y_train_pred_lgbm = lgbm.predict(X_train_scaled)
log_y_test_pred_lgbm = lgbm.predict(X_test_scaled)

r2_train = r2_score(log_y_train, log_y_train_pred_lgbm)
r2_test = r2_score(log_y_test, log_y_test_pred_lgbm)

rmse_train = mean_squared_error(log_y_train, log_y_train_pred_lgbm) ** 0.5
rmse_test = mean_squared_error(log_y_test, log_y_test_pred_lgbm) ** 0.5

print('--------------------------------')
print(f'R² на train: {round(r2_train * 100, 2)}%')
print(f'R² на test: {round(r2_test * 100, 2)}%')
print(f'RMSE на train: {round(rmse_train, 4)}')
print(f'RMSE на test: {round(rmse_test, 4)}')
print('--------------------------------')

--------------------------------
R² на train: 17.3%
R² на test: 10.49%
RMSE на train: 1.4741
RMSE на test: 1.5372
--------------------------------


**Подберём оптимальные параметры LGBM с помощью Optuna:**

In [22]:
import optuna
from optuna.samplers import TPESampler
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score

scaler = StandardScaler().fit(X_train_poly)
X_train_scaled = scaler.transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 100)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    
    lgbm = LGBMRegressor(n_estimators = n_estimators, 
                         max_depth = max_depth, 
                         random_state = 42)
    
    score = cross_val_score(lgbm, X_train_scaled, log_y_train, cv = 3, 
                            scoring = 'neg_mean_squared_error', 
                            n_jobs=-1)
    return -score.mean()
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(sampler = TPESampler(seed = 42), direction = 'minimize')
study.optimize(objective, n_trials = 20)

print(f'Лучшие параметры Optuna: {study.best_params}')

best_lgbm = LGBMRegressor(**study.best_params, random_state = 42)
best_lgbm.fit(X_train_scaled, log_y_train)

log_y_train_pred_lgbm = best_lgbm.predict(X_train_scaled)
log_y_test_pred_lgbm = best_lgbm.predict(X_test_scaled)

r2_train = r2_score(log_y_train, log_y_train_pred_lgbm)
r2_test = r2_score(log_y_test, log_y_test_pred_lgbm)

rmse_train = mean_squared_error(log_y_train, log_y_train_pred_lgbm) ** 0.5
rmse_test = mean_squared_error(log_y_test, log_y_test_pred_lgbm) ** 0.5

print('--------------------------------')
print(f'R² на train: {round(r2_train * 100, 2)}%')
print(f'R² на test: {round(r2_test * 100, 2)}%')
print(f'RMSE на train: {round(rmse_train, 4)}')
print(f'RMSE на test: {round(rmse_test, 4)}')
print('--------------------------------')

Лучшие параметры Optuna: {'n_estimators': 87, 'max_depth': 13}
--------------------------------
R² на train: 37.7%
R² на test: 12.91%
RMSE на train: 1.2794
RMSE на test: 1.5163
--------------------------------


**R² сильно больше на train, чем на test.** Вероятно, в модели наблюдается переобучение.

Попробуем использовать стекинговую модель не только с LGBM, но и с XGBoost и CatBoost:

In [23]:
from sklearn.ensemble import StackingRegressor
#from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
#from sklearn.ensemble import BaggingRegressor
#from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

scaler = StandardScaler().fit(X_train_poly)
X_train_scaled = scaler.transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

stack = StackingRegressor(estimators = [('xgb', xgb.XGBRegressor(n_estimators = 100, 
                                                                 random_state = 42)), 
                                        ('cat', CatBoostRegressor(n_estimators = 100, 
                                                                  verbose = 0, 
                                                                  random_state = 42)),
                                        ('lgbm', LGBMRegressor(n_estimators = 87, max_depth = 13, 
                                                               random_state = 42))], 
                          final_estimator = Ridge())



stack.fit(X_train_scaled, log_y_train)

log_y_train_pred_stack = stack.predict(X_train_scaled)
log_y_test_pred_stack = stack.predict(X_test_scaled)

r2_train = r2_score(log_y_train, log_y_train_pred_stack)
r2_test = r2_score(log_y_test, log_y_test_pred_stack)

rmse_train = mean_squared_error(log_y_train, log_y_train_pred_stack) ** 0.5
rmse_test = mean_squared_error(log_y_test, log_y_test_pred_stack) ** 0.5


print('--------------------------------')
print(f'R² на train: {round(r2_train * 100, 2)}%')
print(f'R² на test: {round(r2_test * 100, 2)}%')
print(f'RMSE на train: {round(rmse_train, 4)}')
print(f'RMSE на test: {round(rmse_test, 4)}')
print('--------------------------------')

--------------------------------
R² на train: 49.03%
R² на test: 13.9%
RMSE на train: 1.1572
RMSE на test: 1.5077
--------------------------------


1. R² значительно вырос на train, при этом на test увеличился на 1%. 
2. Наблюдается переобучение
3. Бустинговые модели лучше работают в данном случае, чем обычная линейная регрессия

## Выводы

В рамках данного этапа мы построили модель для предсказания значений целевой переменной **stream** (логарифм количества прослушиваний музыкальной композиции):

1. При построении обычной линейной регрессии коэффициент детерминации R² у моделей находится в диапазоне 6%-8%, что говорит о плохой объясняющей силе модели. Также, признаки коррелированы между собой, из-за чего в модели присутствует мультиколлинеарность. 

При удалении регрессоров с высоким VIF ***R² на train составил 7.14%, R² на test составил 6.99%***

2. При использовании регуляризации лучше работает Ridge, поскольку Lasso активно зануляет признаки и R² становится отрицательным. При этом относительно обычной регрессии результаты меняются не сильно. Предположительно, основной причиной низкого R² и отсутствия изменений при регуляризации является недостаток важных признаков в данных.


3. Для улучшения модели и её объясняющей силы в модель были добавлены полиномиальные признаки степени 2 для всех регрессоров. При использоваии Ridge R² удалось повысить ***на train до 10.55%, и на test до 9.37%***


4. С использованием бустинговых моделей объясняющая сила значительно улучшилась, однако при этом R² на train сильно выше, чем на test, что говорит о наличии переобучения. **Наилучший результат показала стекинговая модель с использованием LGBM, XGBoost и CatBoost**, ***R² на train составил 49.03%, на test 13.9%***, однако модель явно переобучилась.


5. В рамках текущей работы не удалось построить модель, которая бы качественно предсказывала количество прослушиваний по имеющимся переменным, и при дальнейших исследованиях будет крайне необходимо дополнить анализ и другими характеристиками, *например жанрами, годом выхода композиции и т.д.* Таким образом качество моделей должно значительно возрасти, поскольку в текущей реализации в данных явно не хватает важных регрессоров, из-за чего модель переобучается и не удается повысить объясняющую силу.